In [1]:
import pandas as pd
import json
import numpy as np

# Read the CSV file into a pandas DataFrame
df = pd.read_csv("alltires_combined_mapped.csv")

In [2]:
df = df.loc[:, ["wpid", "productName", "longDescription", "tire_type_GPT", "tire_type_rationale_GPT"]]

In [3]:
df.replace(np.nan, "", inplace=True)
df.rename(
    columns={
        "productName": "product_name",
        "longDescription": "product_description",
        "tire_type_GPT": "llm_label",
        "tire_type_rationale_GPT": "llm_rationale",
    },
    inplace=True,
)

In [4]:
df.head()

,wpid,product_name,product_description,llm_label,llm_rationale
0,3MFDSGOWDYSY,Firestone Super All Traction II 23 R1 15.5-38 ...,<ul><li>Exceptional handling wet and dry</li><...,Agricultural Tires,The Firestone Super All Traction II 23 R1 15.5...
1,6LGF9D1QBAIR,WINTERFORCE 2 UV,"Great Winter Performance, certified with the 3...",Automobile or Light Truck Tires,The product description mentions 'Winter Perfo...
2,4CQJKG37PR9L,Mastercraft Glacier Grip Ii P215/75r15 1,Mastercraft Glacier Grip Ii P215/75r15 1,Automobile or Light Truck Tires,The product name and description indicate that...
3,3U0G2MBGO14O,Goodyear Lt285/70r17 121q D Wrl Duratrac Owl Tl,Goodyear Lt285/70r17 121q D Wrl Duratrac Owl Tl,Automobile or Light Truck Tires,The product name and description indicate that...
4,4ALJ3OP3OR8Y,Hankook ah24 LT11/00R24.5 tire,Hankook ah24 LT11/00R24.5 tire,Automobile or Light Truck Tires,The product name and description indicate that...


In [68]:
df.loc[df["wpid"] == "572AO8P4KQ68"]

,wpid,product_name,product_description,llm_label,llm_rationale
68,572AO8P4KQ68,Telstar Staggered Knobby 25/12-9 Tire,Telstar Staggered Knobby 25/12-9,All-Terrain Vehicle (ATV) Tires,The Telstar Staggered Knobby 25/12-9 tire is d...


In [70]:
df_original[df_original["wpid"] == "572AO8P4KQ68"]

,wpid,item_id,productName,shortDescription,product_medium_description,longDescription,product_category,product_type,tire_type,tire_aspect_ratio,...,wheel_diameter,wheel_diameter_raw_data,wheel_type,tire_type_GPT,tire_type_rationale_GPT,studded_tire_GPT,studded_tire_rationale_GPT,prompt_tokens,completion_tokens,chatGPT_prediction
68,572AO8P4KQ68,714164676,Telstar Staggered Knobby 25/12-9 Tire,Telstar Staggered Knobby 25/12-9,NaN,Telstar Staggered Knobby 25/12-9,"Vehicles, Parts & Accessories",Tires,NaN,NaN,...,9,9.0,NaN,All-Terrain Vehicle (ATV) Tires,The Telstar Staggered Knobby 25/12-9 tire is d...,No,There is no mention of studs or stud compatibi...,381,166,All-Terrain Vehicle (ATV) Tires


In [50]:
np.random.seed(42)

fractions = np.array([0.75, 0.15, 0.1])
# shuffle your input
df = df.sample(frac=1)
# split into 3 parts
train, val, test = np.array_split(df, (fractions[:-1].cumsum() * len(df)).astype(int))

/Users/a0k0b2g/Desktop/Atlassian_Repos/Momentum/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [76]:
choices = [
    "All-Terrain Vehicle (ATV) Tires",
    "Agricultural Tires",
    "Automobile or Light Truck Tires",
    "Bicycle Tires",
    "Forklift Tires",
    "Golf Cart Tires",
    "Lawnmower Tires",
    "Motorcycles/motorized Bicycles Tires",
    "Off-road Tires",
    "Trailer Tires",
    "Commercial Tires",
    "None of the above",
]
# Convert DataFrame to a list of dictionaries
train_data = train.to_dict(orient="records")

for record in train_data:
    # record["choices"] = ["Animals & Pet Supplies", "Apparel & Accessories", "Arts & Entertainment", "Baby & Toddler", "Business & Industrial", "Cameras & Optics", "Electronics", "Food & Beverages", "Furniture", "Hardware", "Health & Beauty", "Home & Garden", "Luggage & Bags", "Mature", "Media", "Office Supplies", "Software", "Sporting Goods", "Toys & Games", "Vehicles & Parts"]
    record["choices"] = choices
# Write the data to a JSONL file
with open("tire_type_train.json", "w") as f:
    for record in train_data:
        json.dump(record, f)
        f.write("\n")

# Convert DataFrame to a list of dictionaries
val_data = val.to_dict(orient="records")

for record in val_data:
    # record["choices"] = ["Animals & Pet Supplies", "Apparel & Accessories", "Arts & Entertainment", "Baby & Toddler", "Business & Industrial", "Cameras & Optics", "Electronics", "Food & Beverages", "Furniture", "Hardware", "Health & Beauty", "Home & Garden", "Luggage & Bags", "Mature", "Media", "Office Supplies", "Software", "Sporting Goods", "Toys & Games", "Vehicles & Parts"]
    record["choices"] = choices
# Write the data to a JSONL file
with open("tire_type_valid.json", "w") as f:
    for record in val_data:
        json.dump(record, f)
        f.write("\n")


# Convert DataFrame to a list of dictionaries
test_data = test.to_dict(orient="records")

for record in test_data:
    # record["choices"] = ["Animals & Pet Supplies", "Apparel & Accessories", "Arts & Entertainment", "Baby & Toddler", "Business & Industrial", "Cameras & Optics", "Electronics", "Food & Beverages", "Furniture", "Hardware", "Health & Beauty", "Home & Garden", "Luggage & Bags", "Mature", "Media", "Office Supplies", "Software", "Sporting Goods", "Toys & Games", "Vehicles & Parts"]
    record["choices"] = choices
# Write the data to a JSONL file
with open("tire_type_test.json", "w") as f:
    for record in test_data:
        json.dump(record, f)
        f.write("\n")

In [6]:
df_original = pd.read_csv('results/combined3.csv')

/var/folders/77/tx60qbb90csc7y3ppwbnj3c80000gn/T/ipykernel_20201/3429452830.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv('results/combined3.csv')


In [ ]:
df_original["chatGPT_prediction"] = ""
# replace chatGPT_prediction column 

In [32]:
train = df 

choices = [
    "All-Terrain Vehicle (ATV) Tires",
    "Agricultural Tires",
    "Automobile or Light Truck Tires",
    "Bicycle Tires",
    "Forklift Tires",
    "Golf Cart Tires",
    "Lawnmower Tires",
    "Motorcycles/motorized Bicycles Tires",
    "Off-road Tires",
    "Trailer Tires",
    "Commercial Tires",
    "None of the above",
]
# Convert DataFrame to a list of dictionaries
train_data = train.to_dict(orient="records")

for record in train_data:
    # record["choices"] = ["Animals & Pet Supplies", "Apparel & Accessories", "Arts & Entertainment", "Baby & Toddler", "Business & Industrial", "Cameras & Optics", "Electronics", "Food & Beverages", "Furniture", "Hardware", "Health & Beauty", "Home & Garden", "Luggage & Bags", "Mature", "Media", "Office Supplies", "Software", "Sporting Goods", "Toys & Games", "Vehicles & Parts"]
    record["choices"] = choices
# Write the data to a JSONL file
with open("tire_type_test.json", "w") as f:
    for record in train_data:
        json.dump(record, f)
        f.write("\n")
